In [ ]:
'''
中央研究院中國文哲研究所
https://www.litphil.sinica.edu.tw/publications/bulletin
'''

'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 加入行為鍊 ActionChain (模擬滑鼠移動、點繫、拖曳、按右鍵出現選單，以及鍵盤輸入文字、按下鍵盤上的按鈕等)
from selenium.webdriver.common.action_chains import ActionChains

# 取得系統時間的工具
from datetime import datetime

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# regular expression 工具
import re

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 隨機亂數(整數)
from random import randint

# 格式化字串輸出
from pprint import pprint as pp

# 處理編碼字串轉換
from urllib import parse

'''
[1] Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
[2] selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
[3] Mouse Hover Action in Selenium
參考網址：https://www.toolsqa.com/selenium-webdriver/mouse-hover-action/
參考網址：https://selenium-python.readthedocs.io/api.html#module-selenium.webdriver.common.action_chains
參考網址：https://python-selenium-zh.readthedocs.io/zh_CN/latest/7.2%20%E8%A1%8C%E4%B8%BA%E9%93%BE/
[4] How to select a drop-down menu value with Selenium using Python?
參考網址：https://stackoverflow.com/questions/7867537/how-to-select-a-drop-down-menu-value-with-selenium-using-python
'''

'''設定'''
# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument('--user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"')  #請求標頭中的使用者代理資訊


# 建立儲存圖片的資料夾，不存在就新增
folderPath = 'sinica'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 下載路徑 (請先在專案目錄下，建立一個新資料夾 files)
download_path = f"C:\\Users\\Owner\\notebooks\\cases\\{folderPath}"

# 指定 chromedriver 檔案的路徑
my_executable_path = './chromedriver.exe'

# Web Driver 變數
driver = None

# 走訪首頁
url = "https://www.litphil.sinica.edu.tw/publications/bulletin"

# 放置資料的變數
listData = []

# JSON 存檔路徑
json_path = "./中央研究院中國文哲研究所集刊.json"


'''程式區域'''
# web driver 初始化
def init():
    global driver
    
    # 使用 Chrome 的 WebDriver
    driver = webdriver.Chrome( 
        options = my_options, 
        executable_path = my_executable_path
    )
    
# 走訪首頁
def visitHomePage():
    # 進入首頁
    driver.get(url)
    
# 走訪各期頁面 (直接走訪分頁取得期刊連結)
def getMainPageLinks(): 
    try:
        # 互動元素的選擇器
        li_pagination_elms_css_selector = 'ul.pagination > li'
        li_data_elms_css_selector = 'ul.publications > li'
        
        # 等待特定元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, li_data_elms_css_selector)
            )
        )
        
        # 取得分頁資料
        li_pagination_elms = driver.find_elements(By.CSS_SELECTOR, li_pagination_elms_css_selector)
        for li_pagination in li_pagination_elms:
            # 按下分頁號碼
            li_pagination.click()
            
            # 等待
            sleep( randint(1,5) )
            
            # 取得集刊資料
            li_data_elms = driver.find_elements(By.CSS_SELECTOR, li_data_elms_css_selector)
            for li_data in li_data_elms:
                # metadata 初始化
                num = title = sub_title = link = '';
                
                # 取得屬性，同時取得期號
                bg_image = li_data.find_element(By.CSS_SELECTOR, "div.cover").get_attribute('bg-image')
                regex = r"\/0*(\d+)\/.+?\.(jpg|png)"
                match = re.search(regex, bg_image)
                if match != None:
                    # 期號
                    num = match[1]
                    
                    # 標題 與 期號(文字)
                    div = li_data.find_element(By.CSS_SELECTOR, "div.name.ng-binding")
                    list_text = div.get_attribute('innerText').split("\n")
                    title = list_text[0]
                    sub_title = list_text[1]
                    
                    # 內頁連結
                    link = f"{url}/{num}"
                    
                    # 整合資料
                    listData.append({
                        "title": title,
                        "sub_title": sub_title,
                        "link": link
                    })
    except TimeoutException:
        print("操作逾時")
        close()

# 取得 PDF 連結
def getPdfLinks():
    try:
        for index, _dict in enumerate(listData):
            # 沒有 sub 屬性，則建立，為了放置細節頁的內容
            if "sub" not in listData[index]:
                listData[index]['sub'] = []

                # 過濾出 pdf 連結
                regex = r"https?:\/\/.+\.pdf"

                # 前往期刊細節頁面
                driver.get(_dict['link'])

                # 等待主要元素出現
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "ol a[href$=pdf]")
                    )
                )

                # 取得所有超連結
                a_elms = driver.find_elements(By.CSS_SELECTOR, "ol a[href$=pdf]")
                if len(a_elms) > 0:
                    for a in a_elms:
                        listData[index]['sub'].append({
                            "pdf_title": a.get_attribute('innerText'), # 或是 a.text
                            "pdf_link": a.get_attribute('href')
                        })
                        
                        # 下載檔案名稱
                        str_pdf_name = a.get_attribute('innerText')
                        str_pdf_name = re.sub(r"\s|/|／|:|──|-|\?|《|》", "_", str_pdf_name)
                        str_pdf_name = parse.unquote(str_pdf_name)
                        
                        # 定義下載連結
                        download_link = f"./{folderPath}/{listData[index]['title']}_{listData[index]['sub_title']}_{str_pdf_name}.pdf"
                        
                        # 下載檔案
                        os.system(f'curl "{a.get_attribute("href")}" -o "{download_link}"')

                        print(f"已下載: {download_link}")
                        
                        # 等待
                        sleep( randint(1,3) )
                        
    except TimeoutException:
        print("元素等待逾時…")
        sleep(3)
        close()

# 關閉瀏覽器
def close():
    driver.quit()
    
# 儲存 JSON
def saveToJson():
    global listData
    with open(f"{folderPath}/{json_path}", "w", encoding='utf-8') as fp:
        fp.write( json.dumps(listData, ensure_ascii=False, indent=4) )

# 讀取 JSON
def loadJson():
    global listData
    with open(f"{folderPath}/{json_path}", "r", encoding="utf-8") as fp:
        listData = json.loads( fp.read() )
        
'''執行區域'''
if __name__ == "__main__":
    # 開始
    init()
    visitHomePage()
    getMainPageLinks()
    getPdfLinks()
    close()
    saveToJson()

curl "https://www.litphil.sinica.edu.tw/public/publications/bulletin/58/%E4%B8%AD%E8%8B%B1%E6%96%87%E5%B0%81%E9%9D%A2%E7%9B%AE%E9%8C%84.pdf" -o "./sinica/中國文哲研究集刊_第五十八期_中英文封面目錄___.pdf"
curl "https://www.litphil.sinica.edu.tw/public/publications/bulletin/58/58-1-50.pdf" -o "./sinica/中國文哲研究集刊_第五十八期_張宗祥校本_說郛_考實__沈暢_.pdf"
curl "https://www.litphil.sinica.edu.tw/public/publications/bulletin/58/58-51-88.pdf" -o "./sinica/中國文哲研究集刊_第五十八期_Dwelling_in_the_Image__How_Should_We_Look_at_Pre_modern_Chinese_Mountain_Water_Paintings___宋灝_.pdf"
curl "https://www.litphil.sinica.edu.tw/public/publications/bulletin/58/58-89-126.pdf" -o "./sinica/中國文哲研究集刊_第五十八期_張栻思想演變問題新探_—以心地工夫意指所向之本體義為考察核心__羅聖堡_.pdf"
curl "https://www.litphil.sinica.edu.tw/public/publications/bulletin/58/58-127-160.pdf" -o "./sinica/中國文哲研究集刊_第五十八期__穀梁傳_注疏合刻與單疏本相關問題考辨__簡逸光_.pdf"
curl "https://www.litphil.sinica.edu.tw/public/publications/bulletin/58/58-161-198.pdf" -o "./sinica/中國文哲研究集刊_第五十八期_書評PDF合併下載___.pdf"
curl "https://www.litphil.si

curl "https://www.litphil.sinica.edu.tw/public/publications/bulletin/51/51-1-37.pdf" -o "./sinica/中國文哲研究集刊_第五十一期_得其氣韻之美_陸時雍論李白古體詩的詩學意義__張俐盈_.pdf"
curl "https://www.litphil.sinica.edu.tw/public/publications/bulletin/51/51-39-64.pdf" -o "./sinica/中國文哲研究集刊_第五十一期_親親相隱，正義與儒家倫理中的道德兩難__王慶節_.pdf"
curl "https://www.litphil.sinica.edu.tw/public/publications/bulletin/51/51-65-94.pdf" -o "./sinica/中國文哲研究集刊_第五十一期_無用與機心的當代意涵_海德格對莊子思想的闡發__鍾振宇_.pdf"
curl "https://www.litphil.sinica.edu.tw/public/publications/bulletin/51/51-95-118.pdf" -o "./sinica/中國文哲研究集刊_第五十一期_解脫軍_現觀莊嚴論釋_：“kiṃ_khalu_hārasya_vihāreṇa__sālakṣaṇyam”_一句中_“hāra”_與_“vihāra”_涵義的探討__宗玉媺_.pdf"
curl "https://www.litphil.sinica.edu.tw/public/publications/bulletin/51/51-119-146%E9%A0%81.pdf" -o "./sinica/中國文哲研究集刊_第五十一期_明清經學關聯性的一種考察_論清代官編_詩經_專著徵引明人的詮解及其他__楊晉龍_.pdf"
curl "https://www.litphil.sinica.edu.tw/public/publications/bulletin/51/51-147-176.pdf" -o "./sinica/中國文哲研究集刊_第五十一期__福音演義：晚清漢語基督教小說的書寫___楊雅惠_.pdf"
curl "https://www.litphil.sinica.edu